In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

Create raw data

In [2]:
from metal.utils import hard_to_soft

N = 1200
X = np.random.random((N,2)) * 2 - 1

Y = np.zeros((N,1))
Y[:,0] = (X[:,0] > X[:,1] + 0.5).astype(int) + 1

X = torch.tensor(X, dtype=torch.float)
Y = torch.tensor(Y, dtype=torch.long)

Separate into splits

In [3]:
X_train = X[:1000]
X_dev   = X[1000:1100]
X_test  = X[1100:]

Y_train = Y[:1000, 0]
Y_dev   = Y[1000:1100]
Y_test  = Y[1100:, 0]

Overwriting batchnorm=True to batchnorm=False
Reaffirming dropout=0.0
Overwriting layer_output_dims=[100, 50] to layer_output_dims=[2]
Overwriting task_head_output_dims=None to task_head_output_dims=2
Sequential(
  (0): Sequential(
    (0): IdentityModule()
  )
  (1): Linear(in_features=2, out_features=2, bias=True)
)


### Logistic Regression

In [10]:
from metal.end_model.baselines import LogisticRegression
em = LogisticRegression(input_dim=2)
em.train(X_train, Y_train, X_dev, Y_dev, n_epochs=40, print_every=5)
em.score(X_test, Y_test)

Overwriting batchnorm=True to batchnorm=False
Reaffirming dropout=0.0
Overwriting layer_output_dims=[100, 50] to layer_output_dims=[2]
Overwriting task_head_output_dims=None to task_head_output_dims=2
Sequential(
  (0): Sequential(
    (0): IdentityModule()
  )
  (1): Linear(in_features=2, out_features=2, bias=True)
)
Reaffirming n_epochs=40
Reaffirming print_every=5
[E:1]	Train Loss: 0.540	Dev score: 0.710
[E:5]	Train Loss: 0.398	Dev score: 0.810
[E:10]	Train Loss: 0.304	Dev score: 0.900
[E:15]	Train Loss: 0.266	Dev score: 0.950
[E:20]	Train Loss: 0.249	Dev score: 0.950
[E:25]	Train Loss: 0.237	Dev score: 0.970
[E:30]	Train Loss: 0.231	Dev score: 0.970
[E:35]	Train Loss: 0.226	Dev score: 0.970
[E:40]	Train Loss: 0.223	Dev score: 0.970
Accuracy: 0.930


0.93

### Multi-layer NN

In [13]:
from metal.end_model import EndModel

em = EndModel(
    seed=1,
    dropout=0.0,
    batchnorm=False,
    layer_output_dims=[2,4,2],
)
em.train(X_train, Y_train, X_dev, Y_dev, n_epochs=40, print_every=5)
em.score(X_test, Y_test)

Overwriting seed=None to seed=1
Reaffirming dropout=0.0
Overwriting batchnorm=True to batchnorm=False
Overwriting layer_output_dims=[100, 50] to layer_output_dims=[2, 4, 2]

Network architecture:
Sequential(
  (0): Sequential(
    (0): IdentityModule()
  )
  (1): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=True)
  )
  (2): Sequential(
    (0): Linear(in_features=4, out_features=2, bias=True)
  )
  (3): Linear(in_features=2, out_features=2, bias=True)
)

Reaffirming n_epochs=40
Reaffirming print_every=5
[E:1]	Train Loss: 0.688	Dev score: 0.710
[E:5]	Train Loss: 0.568	Dev score: 0.710
[E:10]	Train Loss: 0.528	Dev score: 0.710
[E:15]	Train Loss: 0.440	Dev score: 0.720
[E:20]	Train Loss: 0.324	Dev score: 0.910
[E:25]	Train Loss: 0.227	Dev score: 0.940
[E:30]	Train Loss: 0.186	Dev score: 0.980
[E:35]	Train Loss: 0.157	Dev score: 0.980
[E:40]	Train Loss: 0.146	Dev score: 0.990
Accuracy: 0.990


0.99

### Multi-task

In [ ]:
# Y = np.zeros((N,2))
# Y[:,0] = (X[:,0] > X[:,1] + 0.5).astype(int) + 1
# Y[:,1] = (X[:,0] > X[:,1] + 0.25).astype(int) + 1

In [ ]:
# from metal.end_model import EndModel
# from metal.structs import TaskGraph
# from metal.utils import hard_to_soft

# Y_train = [Y[:1000, 0],     Y[:1000, 1]]
# Y_dev   = [Y[1000:1100, 0], Y[1000:1100, 1]]
# Y_test  = [Y[1100:, 0],     Y[1100:, 1]]

# Y_train = [hard_to_soft(Y_t, k=2) for Y_t in Y_train]

# edges = [(0,1)]
# cards = [2,2]
# tg = TaskGraph(edges, cards)

# em = EndModel(
#     label_map=[[1,2],[1,2]], 
#     task_graph=tg,
#     seed=1,
#     dropout=0.0,
#     layer_output_dims=[2,10,4,2],
#     head_layers=[2,3],
#     pass_predictions=True,
# )